# Data


Data we will be using in project 4 are different from our previous projects.

* Monthly patterns data merged with core places data:
  - Under `Grace` workspace, `public/clean-data`.
  - We have been using the weekly patterns data for 4 weeks in March 2020.
  - Now we expand our data by using monthly patterns from 4 months: Jan, Apr, Jul, Oct.
  - In addition, I reduced your workload by merging the core-places data for you already.
  - I also include an additional variable, `poi_cbg`, which is also called *FIPS code*. This is the identifier for Census data.
  - Like in project 1, you will need to clean the data a little, split 'visits_by_day' into dailyvisits, and transform the data from wide to long.
  
* Census data
  - The data I use is under `Grace` workspace, `public/clean-data/cbg.csv`.
  - Complete data under `Grace` workspace, `public/safegraph_open_census_data/data`.
  - Data description: https://www.safegraph.com/blog/beginners-guide-to-census
  - It consists of multiple datasets, each representing survey results of a category. 
    + 01 Age; Sex
    + 02 Race
    + 03 Hispanic or Latino Origin
    + 07 Migration/Residence 1 Year Ago
    + 08 Commuting (Journey to Work); Place of Work
    + 09 Relationship to Householder
    + 11 Household Type; Family Type; Subfamilies
    + 12 Marital Status; Marital History
    + 17 Poverty Status
    + 19 Income
    + 20 Earnings
    + 21 Veteran Status; Period of Military Service
    + 22 Food Stamps/Supplemental Nutrition Assistance Program (SNAP)
    + 23 Employment Status; Work Status Last Year 
    + 24 Industry, Occupation, and Class of Worker
    
    i.e. data `cbg_b01.csv` contains survey results on population age and sex, each row identified by `census_block_group`, which is the *FIPS code*, and is the same as the variable `poi_cbg` in the patterns data above.
  - The variable names consist of letters and numbers. A complete description of what each column refers to is in the file `public/safegraph_open_census_data/metadata/cbg_field_descriptions.csv`, which you can open using excel.
  
  
* US covid cases and death
  - Under `Grace` workspace, `public/clean-data/us-counties.csv`.
  - The data is by county.
  - Recall Safegraph data is by city. To match the US covid data with Safegraph data, use the FIPS code provided.
* For description of other data we are using, refer to the project.

# Part 1: Prepare Data

Question 1a stated, in GRACE, go to `public/clean-data`, download the four datasets of Safegraph dailyvisits for January, April, July, and October 2020 respectively. From each of them, extract a subsample for your city. Concatenate the four subsamples into one, and reshape it into the long form as you have done in Project 1. Each row of your long data should represent a date and a Safegraph place id.

In [1]:
# Modules are imported below:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [39]:
# The 4 months of data is downloaded and concatenated together belowand merged with the core-places data, and census data. 

# months = ['01', '04', '07', '10']
# df = pd.concat([pd.read_csv('data{}.csv'.format(i), 
#                             dtype = {'poi_cbg':'object',
#                                      'naics_code': 'object'}) 
#                 for i in months], 
#                ignore_index = True)
# df.head()

In [40]:
# A subsample for the city Cheyenne is extracted below:

# df = df.loc[df['city'].isin(['Cheyenne']), :]
# df.head()

# subset to a small sample
# df = df.sample(frac=0.01, random_state=1)

In this project, we don't want to work with any missing values.

In real-world, we will first consider if we want to impute the missing values or not.

But since I know that there are only a few (less than 1%) of the data contains missing values, I will simply delete those rows containing missing values. And you can see me doing this constantly throughout the example.

In [2]:
noaa = pd.read_csv('noaa.csv')

In [5]:
# # delete all rows containing missing values
# df = df[~df.isna().any(axis = 1)].copy()

In [6]:
# # define bizgroups
# df.loc[df['naics_code'] == '722511', 'bizgroup'] = '1' # full service restaurant
# df.loc[df['naics_code'].str.startswith('721'), 'bizgroup'] = '2' # accomodation
# df.loc[df['naics_code'].str.startswith('4451'), 'bizgroup'] = '3' # grocery
# df.loc[df['naics_code'] == '92', 'bizgroup'] = '4' # Public Administration

In [7]:
# # clean data as we did in project 1
# df['visits_by_day'] = df['visits_by_day'].str.replace('\[|\]', '')
#
# cols = []
#for i in range(31):
#     v = 'dailyvisits{}'.format(i)
#     cols.append(v)
# df[cols] = df['visits_by_day'].str.split(',', expand = True)
#
# df['id'] = df.index
# mlong = pd.wide_to_long(df, ['dailyvisits'], i = 'id', j = 'dayofmonth')
#
# mlong = mlong[~mlong['dailyvisits'].isnull()].copy()
# mlong.reset_index(inplace = True)
#
# mlong['date'] = pd.to_datetime(mlong['date_range_start']) \
#                + pd.to_timedelta(mlong['dayofmonth'],
#                                  unit = 'd')
#
# mlong['dayofweek'] = mlong['date'].dt.dayofweek
# daynames = ['Mon', 'Tue', 'Wed', 'Thur', 'Fri', 'Sat', 'Sun']
# mlong['dayname'] = mlong['dayofweek'].replace([0,1,2,3,4,5,6], daynames)
# mlong['dayname'] = pd.Categorical(mlong['dayname'], ordered = True, 
#                                  categories=daynames)
# mlong['weekend'] = np.where(mlong['dayname'].isin(['Sat', 'Sun']), 1, 0)

# mlong.drop(columns = 'visits_by_day', inplace = True)

In [8]:
# mlong.to_csv('mlong.csv', index = False)
# Question 1 a stops here.

After cleaning the Safegraph data, I save it as `data.csv` on my computer. So next time, instead of rerun all code above, I simply read in the data I saved.

In [9]:
df = pd.read_csv('mlong.csv',
                dtype = {'poi_cbg':'object',
                          'naics_code': 'object'})
df.head()

,id,dayofmonth,poi_cbg,region,naics_code,safegraph_place_id,date_range_start,city,bizgroup,dailyvisits,date,dayofweek,dayname,weekend
0,277,0,560210014022,WY,722511,sg:9d80429296d44fc0b570ba35884e3574,2020-01-01,Cheyenne,1.0,29,2020-01-01,2,Wed,0
1,3594,0,560210007001,WY,722511,sg:0ccd14ce55944dc99bc847781b323da0,2020-01-01,Cheyenne,1.0,3,2020-01-01,2,Wed,0
2,11671,0,560210006001,WY,445110,sg:5bcfc84e23dd48bebd793e73385a7e59,2020-01-01,Cheyenne,3.0,2,2020-01-01,2,Wed,0
3,12917,0,560210004022,WY,445120,sg:a829beef90cc4ef193f77ef8b3bafe9e,2020-01-01,Cheyenne,3.0,0,2020-01-01,2,Wed,0
4,13938,0,560210002002,WY,722511,sg:6a496f98ab8649b0b655de9147159efb,2020-01-01,Cheyenne,1.0,1,2020-01-01,2,Wed,0


### Census data

Below is how I created the `cbg.csv` data. I picked a few variables that looks interesting to me, and concat them together.

You can work on this data (but do not use all columns for regression. Give reasons on why you choose certain variables).

Or you can include more variables. In the file `public/safegraph_open_census_data/metadata/cbg_field_descriptions.csv`, you can find a description of each variable. Then, add your variable to the data from the corresponding csv file in `public/safegraph_open_census_data/data/`.

* `cbg.csv`:
  - `B01001e1`, `B01001e2` are total population, and total male population.
  - `B01002e1`, `B01002e2`, `B01002e3` are median age (for total population, for male, and for female, respectively).
  - `B02001e2`, `B02001e3`, `B02001e5` are total white, black & asian population.
  - `B11001e1-3` are total number of Household, # of family HH, and # of family HH with married couples, respectively.
  - `B15003e1` is population of 25 yrs and older.
  - `B15003e2-18` are population of high school and below.
  - `B15003e19-25` are population of college and higher.
  - `B19013e1`, `B19301e1`, `B19025e1` are median HH income, per capita income, aggregate HH income, respectively.

In [10]:
# # population


#cbg_b01 = pd.read_csv('cbg_b01.csv',
#                       dtype = {'census_block_group': 'object'})
#cbg_b01['m/f_ratio'] = cbg_b01['B01001e2']/cbg_b01['B01001e26']
#cbg_b01 = cbg_b01.loc[:,['census_block_group', 'B01001e1', 'B01001e2', 
#                          'B01002e1', 'B01002e2', 'B01002e3']]
#total population, male population, median age (total, by male, female)


# # race
#cbg_b02 = pd.read_csv('cbg_b02.csv',
#                       dtype = {'census_block_group': 'object'},
#                      usecols = ['census_block_group','B02001e2', 
#                                 'B02001e3', 'B02001e5'])
# cbg_b02
# #B02001e2, B02001e3, B02001e5 - white, black & asian



# ## HH
#cbg_b11 = pd.read_csv('cbg_b11.csv',
#                       dtype = {'census_block_group': 'object'})
#b11cols = [col for col in cbg_b11.columns if 'B11001e' in col]
#b11cols.append('census_block_group')
#cbg_b11_clean = cbg_b11.loc[:,b11cols]
# # B11001e1-3: total HH,  family HH, family HH with married couples



# ## Education

# # B15003e1 - 25 yrs and older

#cbg_b15 = pd.read_csv('cbg_b15.csv',
#                       dtype = {'census_block_group': 'object'})
#b15cols = [col for col in cbg_b15.columns if 'B15003e' in col]
#b15cols.append('census_block_group')
#cbg_b15_clean = cbg_b15.loc[:,b15cols]
#cbg_b15_clean.head()



# # income
#cbg_b19 = pd.read_csv('cbg_b19.csv',
#                       dtype = {'census_block_group': 'object'},
#                      usecols = ['census_block_group', 'B19013e1', 
#                                 'B19301e1', 'B19025e1'])
# # median hh income, per capita income, aggregate hh income
#cbg_b19.head()

#cbgg = pd.concat([cbg_b01, cbg_b02, cbg_b11_clean, cbg_b15_clean, cbg_b19],
#                 axis = 1, sort = False)
#cbgg = cbg.loc[:,~cbg.columns.duplicated()]
#cbgg.head()
#cbgg.to_csv('cbgg.csv', index = False)

## Choosing my data

Below, I pick several variables that I think may help with predicting dailyvisits.

Although Machine learning is powerful in that it allows the computer to do a lot of the tasks, to get a good modelling result, it is always crucial to think carefully what variables you want to include in your model. Don't let the computer do it all for you. Otherwise, you may get underfitting models because of not including necessary variables, or uninterpretable coefficients because of stacking in too many irrelavent variables.



I skipped an important step here. Before you fit the model, it is always crucial to do some **exploratory data analysis** before choosing variables and fitting the model. For example:

* Look at each variable you pick, plot it alone or with the response variable to see the relationship between the two, whether it is significant enough for you to include it in your model, and whether there is any transformation (log, polynomial, etc.) that can make the relationship more linear.
* Find and kick out outliers.
* Check correlation between explanatory variables. If the correlation is too high, you probably need to drop some variables.



- For example, you might want to consider log transform income data.
- Maybe you want to delete some outliers.
- Read the description of each variable carefully, and delete any multicollinear variables (for instance, there are some variables that add up to another variable).

In [11]:
# Census data we use, including the match id 'census_block_group' and 16 variables
cbgg = pd.read_csv('cbgg.csv',
                 dtype = {'census_block_group': 'object'})
cbgg.head()

,census_block_group,B01001e1,B01001e2,B01002e1,B01002e2,B01002e3,B02001e2,B02001e3,B02001e5,B11001e1,B11001e2,B11001e3,B11001e4,B11001e5,B11001e6,B11001e7,B11001e8,B11001e9,B15003e1,B15003e2,B15003e3,B15003e4,B15003e5,B15003e6,B15003e7,B15003e8,B15003e9,B15003e10,B15003e11,B15003e12,B15003e13,B15003e14,B15003e15,B15003e16,B15003e17,B15003e18,B15003e19,B15003e20,B15003e21,B15003e22,B15003e23,B15003e24,B15003e25,B19013e1,B19025e1,B19301e1
0,010010201001,745,356,34.1,37.1,33.7,585,160,0,284,193,112,81,11,70,91,63,28,474,0,0,0,0,0,0,0,0,23,0,0,53,6,10,11,128,21,13,88,15,60,23,7,16,NaN,14126600.0,20365.0
1,010010201002,1265,639,41.8,39.9,44.8,1083,104,9,456,390,310,80,4,76,66,58,8,824,7,0,0,0,0,0,0,0,0,0,0,27,10,11,9,216,51,21,137,34,156,118,18,9,77813.0,41514300.0,33336.0
2,010010202001,960,534,38.2,23.6,45.0,361,568,0,386,244,145,99,6,93,142,133,9,602,9,0,0,0,0,0,4,15,0,0,6,17,4,63,3,211,51,70,64,15,54,7,9,0,25179.0,15608200.0,17047.0
3,010010202002,1236,634,39.7,33.5,43.3,615,571,24,452,286,184,102,9,93,166,153,13,861,7,0,0,0,0,0,0,0,6,6,16,18,48,28,24,255,59,63,90,74,113,39,9,6,45104.0,25847500.0,21400.0
4,010010203001,2364,1125,34.9,34.1,39.3,1481,515,27,824,560,435,125,0,125,264,228,36,1648,46,0,0,18,0,0,0,0,0,5,5,39,42,33,20,536,40,122,264,112,205,133,10,18,55222.0,50827900.0,23106.0


In [12]:
# I include two variables that I create
# B15003e2-18: high school and below
# B15003e19-25: college and higher
cbgg['EDU_under'] = cbgg.loc[:, 'B15003e2':'B15003e16'].sum(axis = 1) / cbgg['B15003e1']
cbgg['EDU_higher'] = cbgg.loc[:, 'B15003e21':'B15003e25'].sum(axis = 1) / cbgg['B15003e1']

cbg1 = cbgg.loc[:, ['census_block_group','B01001e1', 'B01001e2', 'B01002e1', 
               'B01002e2','B01002e3', 'B02001e2', 'B02001e3', 'B02001e5', 
               'B11001e1', 'B11001e2', 'B11001e3', 
               'EDU_under', 'EDU_higher', 'B19013e1', 'B19025e1', 'B19301e1']]
cbg1.head()

,census_block_group,B01001e1,B01001e2,B01002e1,B01002e2,B01002e3,B02001e2,B02001e3,B02001e5,B11001e1,B11001e2,B11001e3,EDU_under,EDU_higher,B19013e1,B19025e1,B19301e1
0,010010201001,745,356,34.1,37.1,33.7,585,160,0,284,193,112,0.217300,0.255274,NaN,14126600.0,20365.0
1,010010201002,1265,639,41.8,39.9,44.8,1083,104,9,456,390,310,0.077670,0.406553,77813.0,41514300.0,33336.0
2,010010202001,960,534,38.2,23.6,45.0,361,568,0,386,244,145,0.200997,0.141196,25179.0,15608200.0,17047.0
3,010010202002,1236,634,39.7,33.5,43.3,615,571,24,452,286,184,0.177700,0.279907,45104.0,25847500.0,21400.0
4,010010203001,2364,1125,34.9,34.1,39.3,1481,515,27,824,560,435,0.126214,0.290049,55222.0,50827900.0,23106.0


In [13]:
cbg1['B01001e1'].mean(axis = 0)

1461.82163815679

In your project 4, you are given more variable choices: covid cases, weather, etc. Remember you can also explore the full census data I uploaded on GRACE, under `public/safegraph_open_census_data/data`, and add whatever variable you believ is important.

Now to get the final data, we will merge this census data with the original long data. I call the new data `df2`.

In [14]:
# merge data
df2 = pd.merge(df, cbg1, left_on = 'poi_cbg', right_on = 'census_block_group', how = 'inner')
df2.drop(columns = 'census_block_group', inplace = True)

In our case, there is no missing census FIPS code, so when you merge, there should be no row in `df` left over that is not merged into `df2`. Do check this before you proceed.

In [15]:
# double-check: all rows in df should be merged into df2
print(len(df) - len(df2))

0


In [16]:
# drop all rows with missing values
df2 = df2[~df2.isna().any(axis=1)].copy()

In [17]:
df2['month'] = pd.to_datetime(df2['date']).dt.month.astype('str')
df2['bizgroup'] = df2['bizgroup'].astype('str')

From this data, we obtain the X (set of explanatory variables) and y (response variable) for our regressions.

In [18]:
X = df2.loc[:,['weekend','region', 'bizgroup', 'dayname', 'month',
                             'B01001e1', 'B01001e2', 'B01002e1', 
                             'B01002e2', 'B01002e3', 'B02001e2',
                             'B02001e3', 'B02001e5', 'B11001e1', 
                             'B11001e2', 'B11001e3', 
                             'EDU_under','EDU_higher', 
                             'B19013e1', 'B19025e1', 'B19301e1']]
y = np.log(df2.loc[:, 'dailyvisits'] + 1)

For Ridge or Lasso, we need to make sure the X variables are all numerical. That is, we need to get dummies for all categorical variables.

In [19]:
X.dtypes

weekend         int64
region         object
bizgroup       object
dayname        object
month          object
B01001e1        int64
B01001e2        int64
B01002e1      float64
B01002e2      float64
B01002e3      float64
B02001e2        int64
B02001e3        int64
B02001e5        int64
B11001e1        int64
B11001e2        int64
B11001e3        int64
EDU_under     float64
EDU_higher    float64
B19013e1      float64
B19025e1      float64
B19301e1      float64
dtype: object

In [20]:
# get dummies for all categorical variables
X = pd.get_dummies(data = X, drop_first = True)
X.head()

,weekend,B01001e1,B01001e2,B01002e1,B01002e2,B01002e3,B02001e2,B02001e3,B02001e5,B11001e1,B11001e2,B11001e3,EDU_under,EDU_higher,B19013e1,B19025e1,B19301e1,region_WY,bizgroup_2.0,bizgroup_3.0,dayname_Mon,dayname_Sat,dayname_Sun,dayname_Thur,dayname_Tue,dayname_Wed,month_10,month_4,month_7
0,0,1819,826,37.3,35.1,43.8,1675,12,83,688,335,279,0.075806,0.504032,59808.0,43550200.0,26696.0,1,0,0,0,0,0,0,0,1,0,0,0
1,0,1819,826,37.3,35.1,43.8,1675,12,83,688,335,279,0.075806,0.504032,59808.0,43550200.0,26696.0,1,0,0,0,0,0,0,0,1,0,0,0
2,0,1819,826,37.3,35.1,43.8,1675,12,83,688,335,279,0.075806,0.504032,59808.0,43550200.0,26696.0,1,0,0,0,0,0,0,0,1,0,0,0
3,0,1819,826,37.3,35.1,43.8,1675,12,83,688,335,279,0.075806,0.504032,59808.0,43550200.0,26696.0,1,0,0,0,0,0,0,0,1,0,0,0
4,0,1819,826,37.3,35.1,43.8,1675,12,83,688,335,279,0.075806,0.504032,59808.0,43550200.0,26696.0,1,0,1,0,0,0,0,0,1,0,0,0


# Modelling

* The package we will be using is called `scikit-learn`, also named `sklearn` in python. This is the most common package in ML in python (at least for these regular ML algorithms).

The first thing I will do is to split our data into training and testing data. The train_test_split() function does this for us. By default, if we do not specify train_size or test_size, the function will split the data randomly, with $\approx$ 75% of the data in the train data, and $\approx$ 25% in the test data.

Remember to specify `random_state = `, so that you don't get a different split every time to run this line.

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state = 1)

For Ridge and Lasso, recall that in the loss function, there is a penalty added to the coefficients. Imagine that we have two variables, one has values in the range [0,1], another has values all over the range [-100000, 100000]. And suppose the two variables are of similar importance to our response variables. Then, the coefficient in front of the second variable is considerably smaller (maybe 100000 times smaller) than the coefficient in front of the first variable. In this case, the penalty added to the first variable is significantly larger, which is unfair.

Therefore, we always normalize or standardize the explanatory variables before we fit the model.

I am using MinMaxScalar for normalization.

$$x_{scaled} = \frac{x - min(x)}{max(x) - min(x)}$$

The scaled data is squeezed inside the range $[0,1]$.

The benefit of normalization is that all dummy variables remain in the nice 0/1 format.

In [22]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Linear regression

Let's first run a linear regression on the variables we choose, so we can make some comparisons later on.

In [23]:
import statsmodels.api as sm
X2_train = sm.add_constant(X_train)
results = sm.OLS(y_train,X2_train.astype(float)).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            dailyvisits   R-squared:                       0.197
Model:                            OLS   Adj. R-squared:                  0.195
Method:                 Least Squares   F-statistic:                     78.64
Date:                Fri, 16 Apr 2021   Prob (F-statistic):               0.00
Time:                        23:14:12   Log-Likelihood:                -12916.
No. Observations:                8989   AIC:                         2.589e+04
Df Residuals:                    8960   BIC:                         2.610e+04
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            7.1454      0.344     20.752      0.000       6.470       7.820
weekend         -0.2441      0.023    -10.630      0.000      -0.289      -0.199
B01001e1        -0.0090      0.001    -17.201      0.000      -0.010      -0.008
B01001e2         0.0028      0.000      8.642      0.000       0.002       0.003
B01002e1        -0.0064      0.008     -0.759      0.448      -0.023       0.010
B01002e2        -0.0228      0.006     -3.956      0.000      -0.034      -0.012
B01002e3         0.0140      0.004      3.186      0.001       0.005       0.023
B02001e2         0.0029      0.000     11.033      0.000       0.002       0.003
B02001e3         0.0018      0.001      2.884      0.004       0.001       0.003
B02001e5         0.0195      0.001     21.559      0.000       0.018       0.021
B11001e1         0.0048      0.000     12.239      0.000       0.004       0.006
B11001e2         0.0124      0.001     23.325      0.000       0.011       0.013
B11001e3        -0.0060      0.001     -9.977      0.000      -0.007      -0.005
EDU_under        0.0250      0.537      0.046      0.963      -1.028       1.078
EDU_higher       2.8135      0.287      9.805      0.000       2.251       3.376
B19013e1     -5.643e-06    2.1e-06     -2.689      0.007   -9.76e-06   -1.53e-06
B19025e1       3.95e-08   7.54e-09      5.238      0.000    2.47e-08    5.43e-08
B19301e1     -9.635e-05   9.65e-06     -9.982      0.000      -0.000   -7.74e-05
region_WY       -2.5326      0.124    -20.462      0.000      -2.775      -2.290
bizgroup_2.0    -1.1167      0.107    -10.405      0.000      -1.327      -0.906
bizgroup_3.0    -0.1865      0.035     -5.337      0.000      -0.255      -0.118
dayname_Mon     -0.2940      0.041     -7.204      0.000      -0.374      -0.214
dayname_Sat      0.0399      0.024      1.688      0.092      -0.006       0.086
dayname_Sun     -0.2840      0.024    -11.831      0.000      -0.331      -0.237
dayname_Thur    -0.1438      0.038     -3.773      0.000      -0.219      -0.069
dayname_Tue     -0.2356      0.040     -5.843      0.000      -0.315      -0.157
dayname_Wed     -0.1720      0.039     -4.455      0.000      -0.248      -0.096
month_10        -0.1944      0.031     -6.186      0.000      -0.256      -0.133
month_4         -0.8211      0.030    -27.690      0.000      -0.879      -0.763
month_7         -0.2732      0.030     -9.057      0.000      -0.332      -0.214
==============================================================================
Omnibus:                       53.956   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               39.315
Skew:                           0.045   Prob(JB):                     2.90e-09
Kurtosis:                       2.689   Cond. No.                     1.14e+16
===============

### * A short digression: another way of running linear regression, using scikit-learn package.

In [24]:
from sklearn.linear_model import LinearRegression

linreg = LinearRegression().fit(X_train, y_train)

coef = linreg.coef_

print('R-squared score (training): {:.3f}'.format(linreg.score(X_train, y_train)))
print('R-squared score (test): {:.3f}'.format(linreg.score(X_test, y_test)))

R-squared score (training): 0.197
R-squared score (test): 0.209


In [25]:
# coef_sorted=sorted(coef, key = abs, reverse = True)
# for i, j in enumerate(coef_sorted):
#     print('{var} {coef:.5f}'.format(var = X.columns[i], coef = j))

## Ridge regression

In [26]:
# fit the ridge linear regression model
from sklearn.linear_model import Ridge
linridge = Ridge(alpha=0.1).fit(X_train_scaled, y_train)

print('Ridge regression coef:\n{}'.format(linridge.coef_))
print('R-squared score (training): {:.3f}'.format(linridge.score(X_train_scaled, y_train)))
print('R-squared score (test): {:.3f}'.format(linridge.score(X_test_scaled, y_test)))
print('Number of non-zero features: {}'.format(np.sum(linridge.coef_ != 0)))

Ridge regression coef:
[-2.43832975e-01 -2.11298682e+01  2.60729041e+00 -1.66683887e-02
 -7.81922023e-01  3.99112396e-01  5.89192838e+00  1.87311509e-01
  1.39333475e+00  4.37343680e+00  1.02072498e+01 -3.58956996e+00
 -9.05069485e-02  1.19614321e+00 -6.84406688e-01  2.47115978e+00
 -2.41530474e+00 -2.06663355e+00 -1.06021207e+00 -1.52419898e-01
 -2.95092863e-01  3.92707649e-02 -2.83103739e-01 -1.44368508e-01
 -2.35795049e-01 -1.71128728e-01 -2.01158424e-01 -8.23258981e-01
 -2.81137816e-01]
R-squared score (training): 0.195
R-squared score (test): 0.209
Number of non-zero features: 29


Now, we also want to know which variables are the most important ones in our model.

In [27]:
pip install eli5

Note: you may need to restart the kernel to use updated packages.


In [28]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(linridge).fit(X_test_scaled, y_test)
eli5.show_weights(perm)

Weight,Feature
24.6399 ± 0.9226,x1
5.9456 ± 0.4146,x10
2.0206 ± 0.0687,x6
1.1168 ± 0.0582,x9
0.6428 ± 0.0266,x11
0.4110 ± 0.0271,x8
0.4024 ± 0.0435,x15
0.3459 ± 0.0262,x2
0.2178 ± 0.0204,x16
0.2112 ± 0.0137,x27


## Lasso

In [29]:
from sklearn.linear_model import Lasso

linlasso = Lasso(alpha=0.0001, max_iter = 10000).fit(X_train_scaled, y_train)

print('Non-zero features: {}'.format(np.sum(linlasso.coef_ != 0)))
print('R-squared score (training): {:.3f}'.format(linlasso.score(X_train_scaled, y_train)))
print('R-squared score (test): {:.3f}\n'.format(linlasso.score(X_test_scaled, y_test)))
print('Features with non-zero weight (sorted by absolute magnitude):')

for e in sorted (list(zip(list(X), linlasso.coef_)),
                key = lambda e: -abs(e[1])):
    if e[1] != 0:
        print('\t{}, {:.3f}'.format(e[0], e[1]))

Non-zero features: 27
R-squared score (training): 0.196
R-squared score (test): 0.208

Features with non-zero weight (sorted by absolute magnitude):
	B01001e1, -22.408
	B11001e2, 10.526
	B02001e2, 6.480
	B11001e1, 4.689
	B11001e3, -3.616
	B01001e2, 2.810
	B19301e1, -2.490
	B19025e1, 2.341
	region_WY, -2.131
	B02001e5, 1.441
	EDU_higher, 1.195
	bizgroup_2.0, -1.052
	B01002e2, -0.846
	month_4, -0.822
	B19013e1, -0.605
	B01002e3, 0.383
	dayname_Sun, -0.322
	dayname_Mon, -0.291
	month_7, -0.278
	dayname_Tue, -0.232
	weekend, -0.202
	month_10, -0.198
	B02001e3, 0.197
	dayname_Wed, -0.167
	bizgroup_3.0, -0.158
	dayname_Thur, -0.140
	EDU_under, -0.077


We can compare the fitting results across different choices of alpha parameter:

In [30]:
for alpha in [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 5]:
    linlasso = Lasso(alpha=alpha, max_iter = 10000).fit(X_train_scaled, y_train)
    r2_train = linlasso.score(X_train_scaled, y_train)
    r2_test = linlasso.score(X_test_scaled, y_test)
    
    print('Alpha = {}\nFeatures kept: {}, r-squared training: {:.3f}, \
r-squared test: {:.3f}\n'
         .format(alpha, np.sum(linlasso.coef_ != 0), r2_train, r2_test))

Alpha = 1e-05
Features kept: 28, r-squared training: 0.197, r-squared test: 0.209

Alpha = 0.0001
Features kept: 27, r-squared training: 0.196, r-squared test: 0.208

Alpha = 0.001
Features kept: 25, r-squared training: 0.173, r-squared test: 0.191

Alpha = 0.01
Features kept: 12, r-squared training: 0.109, r-squared test: 0.127

Alpha = 0.1
Features kept: 1, r-squared training: 0.029, r-squared test: 0.032

Alpha = 1
Features kept: 0, r-squared training: 0.000, r-squared test: -0.000

Alpha = 5
Features kept: 0, r-squared training: 0.000, r-squared test: -0.000



### Underfitting and Overfitting

* Underfitting: when you can have better predictions, but you don't, because of choosing the wrong parameters.
* Overfitting: r2 test is significantly lower than r2 train

So it's important to choose a parameter that yields high and similar training and testing scores.

### * More about choosing parameters

The most common practice in ML to choose parameters is call Gridsearch. 

Basically, it is what we are doing above, only that we may use hundreds or even thousands of possible alphas, and pick the one that yields the highest score.

And to make this process more robust, we normally use cross-validation.

In [31]:
from sklearn.linear_model import RidgeCV

n_alphas = 50
alphas = np.logspace(-5, 2, n_alphas)

linridge_cv = RidgeCV(alphas=alphas).fit(X_train_scaled, y_train)
linridge_cv.alpha_

0.0013894954943731374

### Comparing Ridge and Lasso

In [32]:
# I'm using a small subset of the data above to speed up the process.
df3 = df2.sample(frac=0.1, random_state=1)
X3 = df3.loc[:,['weekend','region', 'bizgroup', 'dayname', 'month',
                             'B01001e1', 'B01001e2', 'B01002e1', 
                             'B01002e2', 'B01002e3', 'B02001e2',
                             'B02001e3', 'B02001e5', 'B11001e1', 
                             'B11001e2', 'B11001e3', 
                             'EDU_under','EDU_higher', 
                             'B19013e1', 'B19025e1', 'B19301e1']]
y3 = np.log(df3.loc[:, 'dailyvisits'] + 1)
X3['bizgroup'] = X3['bizgroup'].astype('str')
# get dummies for all categorical variables
X3 = pd.get_dummies(data = X3, drop_first = True)
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3,
                                                    random_state = 1)
X3_train_scaled = scaler.fit_transform(X3_train)
X3_test_scaled = scaler.transform(X3_test)

In [33]:
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib notebook

n_alphas = 50
alphas = np.logspace(-5, 2, n_alphas)

coefs_ridge = []
for alpha in alphas:
    ridge = Ridge(alpha=alpha).fit(X3_train_scaled, y3_train)
    coefs_ridge.append(ridge.coef_)

first10vars_ridge = np.array(coefs_ridge)[:, :10]
plt.figure()
plt.plot(np.log(alphas), first10vars_ridge);
plt.xlabel('log(alpha)')
plt.title('Ridge coefficients with increasing alpha');

<IPython.core.display.Javascript object>

From the above plot, we see that for the smaller alphas we picked, ridge regression does not shrink the coefficients at all. As alpha increases, the coefficients are shrinked gradually, getting infinitely close to 0, but none reaches 0.


Our example also has a special case: we don't have any strong variable that contributes significantly to the prediction of the y variable compared to other variables. Therefore, Ridge shrinkage can do little to improve our model. This shows the importance of picking variables before fitting the model.

In [34]:
n_alphas = 80
alphas = np.logspace(-5, -1, n_alphas)

coefs = []

for alpha in alphas:
    lasso = Lasso(alpha=alpha, max_iter = 10000).fit(X3_train_scaled, y3_train)
    coefs.append(lasso.coef_)
    
first10vars = np.array(coefs)[:, :10]
plt.figure()
plt.plot(np.log(alphas), first10vars)
plt.xlabel('log(alpha)')
plt.title('Lasso coefficients with increasing alpha');

<IPython.core.display.Javascript object>

We see from the plot for Lasso coefficients that some coefficients shrinks quickly and reaches 0.

This means Lasso helps us did some feature selections.

Overall, if it is for prediction purpose, and we believe all variables we included are more or less relevant to predicting y, so that we don't want to get rid of any one of them, we often use Ridge regression.

If we are interested in feature selection, and in particular, if we have more variables than observations, Lasso is very useful in fitting a legit model.

It is not always the case that Ridge outperforms Lasso in prediction.
Lasso can sometimes yield better predictions.

# Some notes on this example

Clearly, the particular models we are using in this example is not a good fit for our prediction goal. This is due to multiple reasons:

* As I mentioned, the variables I pick are not very related to the response variable, so shrinkage method cannot introduce meaningful penalties. If I include more relavant variables (i.e. city, covid cases, etc.), I will get a better prediction.
* I skipped the exploratory data analysis. If we examine each variable, conduct necessary transformations, pick out outliers, etc., we should result in a better prediction.
* Instead of regularization methods, other ML methods like random forest may be more suitable.

For instance, since we know that a lot of the X variables do not have a linear relationship with the y variable, if we include 2nd order polynomials to the model, it gives better fit.

In [35]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_poly, y,
                                                   random_state = 0)
linreg = LinearRegression().fit(X_train, y_train)

print('(poly deg 2) R-squared score (training): {:.3f}'
     .format(linreg.score(X_train, y_train)))
print('(poly deg 2) R-squared score (test): {:.3f}\n'
     .format(linreg.score(X_test, y_test)))

(poly deg 2) R-squared score (training): 0.405
(poly deg 2) R-squared score (test): 0.403



Or, the regression tree below gives a much higher prediction accuracy, since in decision tree, we discard the linear relationship assumption.

Note that this result isn't very robust. It's just an illustration of how different ML algorithm helps with our prediction goal.

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor 

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state = 1)

m1 = DecisionTreeRegressor(min_samples_split = 80, random_state = 1).fit(X_train, y_train)

print('R-squared score (training): {:.3f}'.format(m1.score(X_train, y_train)))
print('R-squared score (test): {:.3f}\n'.format(m1.score(X_test, y_test)))

R-squared score (training): 0.426
R-squared score (test): 0.398



In [37]:
# The plot below shows how nonlinear one of our X variable is. If we tranform it 
# before fitting the model, our coefficients will become more meaningful.
import seaborn as sns
plt.figure(figsize = (4,3))
sns.distplot(sorted(X_train_scaled[:,1]))
plt.gca().set_title('Distribution of B11001e1');

<IPython.core.display.Javascript object>

C:\ProgramData\Anaconda3\lib\site-packages\seaborn\distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


Question 1b (2 points): In GRACE, go to `public/clean-data`, download the data for Census demographics. Merge the long Safegraph dailyvisits data (you have created in Question 1a) with the demographic data, via the key `poi_cbg` in the long sample, and `census_block_group` in the demographic data (both variables represent the 12-digit FIPS code of census block group). Only keep the matched observations. Drop the variable `census_block_group`. From now on, I will refer to this merged data as your “LocalSafegraph” data.

How many observations do you have in your “LocalSafegraph”? How many unique places are there? How many unique NAICS codes are there? What type of places do these NAICS codes represent? How many unique places do you have for each NACIS code? What is the average population for each observation in your “LocalSafeGraph” data?

 

In “LocalSafegraph”, `poi_cbg’ represents the 12-digit FIPS code for each Safegraph place. Define a separate string variable `FIPS_5digits’ as the first five digits of `poi_cbg’. How many unique values of `FIPS_5digits’ do you have in “LocalSafegraph”? Ideally, you should have only one value of `FIPS_5digits’, which represents the state-county your city locates in. If you have more than one value of `FIPS_5digits’, use the most frequent one as the state-county for your city. Write down the value of FIPS_5digits for your city, you will need that later on.

# what do you mean by what is the average population for each observation (as defined: check 4:01: use len()

How many unique is asking for how many unique place ID.

Question 1c (1 point): In GRACE, go to `public/clean-data`, download the dataset of COVID-19 cases as collected by NY Times. Extract the relevant observations for your city using the value of FIPS_5digits you have found in Question 1b. I will refer to this data as your “LocalCOVIDcases”.

 

How many observations are there in your “LocalCOVIDcases”? What is the minimum date? What is the maximum date? What is the average number of daily cases? What is the average number of daily deaths?

Question 1d (2 point): From this NOAA page (Links to an external site.) find daily weather data for one station in or near your city, between 2020-1-1 and 2020-12-31. I will refer to this dataset as your “LocalWeather” data.

 

How many observations are there in your “LocalWeather”? What is the average air temperature? What is the average precipitation?

# Part 2: Merge data for final analysis sample

Question 2a (2 points): Now merge your “LocalSafegraph” with “LocalCovidcases” and “LocalWeather” by date. Keep all dates that appear in “LocalSafegraph”, even if they do not appear in “LocalCovidcases” or “LocalWeather”. Do not include dates not showing up in “LocalSafegraph”. I will refer to the merged data as “LocalMerged”.

 

How many dates in your “LocalMerged” have missing values in the daily count of COVID cases and COVID deaths? Impute these missing values as COVID cases equal to 0 and COVID deaths equal to 0. (NY Times only report COVID information after a COVID case started to occur in a local county.) After the imputation, what are the average daily COVID cases and daily COVID deaths in your “LocalMerged”?

 

Do you have any date that has missing values in air temperature or precipitation? If so, drop these dates from “LocalMerged”.  How many observations do you have now in “LocalMerged”?

Question 2b (2 points): Use online search engines to find COVID related policies for your city or state. Pay special attention to type and timing of policies (e.g. shutdown and different phases of reopening). Create one or multiple variables to describe these policies. For example, if your city/state was shutdown at 2020-03-15, partially reopened on 2020-4-15, and full reopened on 2020-6-25, you can define Shutdown equal to one if the date is between 2020-03-15 and 2020-04-14, zero otherwise; Reopen1 equal to one if the date is between 2020-04-15 and 2020-06-24, zero otherwise; and Reopen2 equal to one if the date is on or after 2020-06-25.

 

How many policy variables do you create? What is their specific definition? Justify why you choose them this way.

Question 2c (2 points): Now define your dependent variable as Log(DailyVisits+1). Define the list of your potential right-hand variables, which should include some industry indicators (generated from NAICS codes), day of week indicators (Monday, Tuesday …, Sunday), some demographics, some weather variables, some variables for COVID cases and deaths, and at least one variable describing COVID-related policies. Feel free to add new variables to this list, as long as you articulate a reason for their relevance.

 

Summarize all these variables in a table with the number of observations, mean, median, standard deviation, minimum and maximum. If some variables are categorical and thus cannot appear in this summary table, create separate table(s) to describe the frequency of each value in such categorical variable(s).

 

In [38]:
Part 3: Linear regression, LASSO and Ridge

SyntaxError: invalid syntax (<ipython-input-38-4c093f2e2490>, line 1)

Question 3a (2 points): Choose a list of variables to predict log(dailyvisits+1). The list you choose could be a subset of the potential right-hand variables you have identified in Question 2c. I will refer to these variables as “features”.

 

Use the Python command “train_test_split()” to separate your “LocalMerged” into a training sample and a test sample.

 

Use function MinMaxScalar() to standardize all the “features” you choose, in both the training and test samples.

 

How many observations are there in the training sample? How many in the test sample? Choose at least two variables to comment on the difference between the two samples. For example, are they different in the months covered? Are they different in weather? Feel free to use other variables for this comparison.

Question 3b (2 points): Using Log(dailyvisits+1) and the standardized features in Question 3a, run a linear regression in your training sample. Report the coefficient estimates and R-squares.

 

What “features” have statistically significant coefficients (with p-value<0.05)? Among these significant “features”, which has a positive coefficient and which has a negative coefficient? Comment on whether their signs and magnitude confirm your prior.

 

If you extend the model to predict log(dailyvisits+1) in the test sample, what is the R-squares in the test sample?

Question 3c (2 points): Using Log(dailyvisits+1) and the standardized features in Question 3a, run LASSO in your training sample.

 

Try at least three different values for the penalty parameter. For each value you have tried, report the coefficient estimates and R-squares; also extend the model to predict log(dailyvisits+1) in the test sample, and report the R-squares in the test sample.

 

Among the penalty parameters you have tried, which one is most appropriate? Explain how you arrive at this choice.

 

In the LASSO model with your preferred choice of the penalty parameter, what “features” have zero (or very close to zero) coefficients? What and how many “features” have non-zero coefficients? Comment on whether their signs and magnitude confirm your prior. What is the R-squares when you apply this model to the test sample? Is it higher or lower than what you find in question 3b for linear regression?

 

Use the random permutation method to identify the five most important features in the above LASSO model. What are they? Comment on whether the order of their importance is consistent with your prior.

 

Question 3d (2 points): Using Log(dailyvisits+1) and the standardized features in Question 3a, run Ridge in your training sample.

 

Try at least three different values for the penalty parameter. For each value you have tried, report the coefficient estimates and R-squares; also extend the model to predict log(dailyvisits+1) in the test sample, and report the R-squares in the test sample.

 

Among the penalty parameters you have tried, which one is most appropriate? Explain how you arrive at this choice.

 

In the Ridge model with your preferred choice of the penalty parameter, what “features” have zero (or very close to zero) coefficients? What and how many “features” have non-zero coefficients? Comment on whether their signs and magnitude confirm your prior. What is the R-squares when you apply this model to the test sample? Is it higher or lower than what you find in question 3b for linear regression and question 3c for LASSO?

 

Use the random permutation method to identify the five most important features in the above Ridge model. What are they? Comment on whether the order of their importance is consistent with your prior. Does Ridge identify the same five most important features as LASSO?